In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, Checkbox

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def gabor(x, y, x0, y0, theta, sigma, f, phi, gamma):
    xp = (x - x0) * np.cos(theta) + (y - y0) * np.sin(theta)
    yp = -(x - x0) * np.sin(theta) + (y - y0) * np.cos(theta)
    envelope = np.exp(-(xp**2 + gamma**2 * yp**2) / (2 * sigma**2))
    carrier = np.cos(2 * np.pi * f * xp + phi)
    return envelope * carrier

In [3]:
def draw_sigma(x0, y0, sigma, theta):
    # Drawing sigma as a vector
    end_x = x0 + sigma * np.cos(theta)
    end_y = y0 + sigma * np.sin(theta)
    plt.arrow(x0, y0, end_x - x0, end_y - y0,
              head_width=0.5, head_length=0.5, fc='red', ec='red', length_includes_head=True)
    plt.text(end_x, end_y, r'$\sigma$', fontsize=12, color='red')

In [4]:
def plot_gabor(x0=0, y0=0, theta=0, sigma=1, f=0.5, phi=0, gamma=1, draw_parameters=False):
    x = np.linspace(-10, 10, 500)
    y = np.linspace(-10, 10, 500)
    X, Y = np.meshgrid(x, y)
    G = gabor(X, Y, x0, y0, theta, sigma, f, phi, gamma)

    plt.figure(figsize=(6, 6))
    plt.imshow(G, extent=(-10, 10, -10, 10), origin='lower')
    plt.colorbar()

    if draw_parameters:
        draw_sigma(x0, y0, sigma, theta)

    plt.show()

In [12]:
interactive_plot = interactive(plot_gabor,
                               x0=FloatSlider(min=-10, max=10, step=0.5, value=0, description='$x_0$:'),
                               y0=FloatSlider(min=-10, max=10, step=0.5, value=0, description='$y_0$:'),
                               theta=FloatSlider(min=0, max=np.pi, step=np.pi/16, value=0, description=r'$\theta$ (Theta):'),
                               sigma=FloatSlider(min=0.1, max=10, step=0.1, value=1, description=r'$\sigma$ (Sigma):'),
                               f=FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='$f$ (Freq.):'),
                               phi=FloatSlider(min=0, max=2*np.pi, step=np.pi/16, value=0, description=r'$\phi$ (Phi):'),
                               gamma=FloatSlider(min=0.1, max=10, step=0.1, value=1, description=r'$\gamma$ (Gamma):'),
                               draw_parameters=Checkbox(value=False, description='Draw parameters'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$x_0$:', max=10.0, min=-10.0, step=0.5), FloatSlider…